In [13]:
from PyPDF2 import PdfReader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
import os
from dotenv import load_dotenv
from langchain_core.documents import Document

load_dotenv()


True

In [14]:
# Initialize Pinecone
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")

In [3]:
print(OPENWEATHER_API_KEY)

4d21dd8126cf6d023afaf25a027ec5b2


In [24]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)
if PINECONE_INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    )

In [25]:
# Function to read PDF and create documents and upload document into vector store
from langchain_pinecone import PineconeVectorStore
embeddings = OpenAIEmbeddings()
def read_pdf(file_path):
    reader = PdfReader(file_path)
    texts = []
    for page in reader.pages:
        texts.append(page.extract_text())
    return texts

# Import PDF into vector store
pdf_texts = read_pdf('data.pdf')
docs = [Document(page_content=text, metadata={}) for text in pdf_texts]
docsearch = PineconeVectorStore.from_documents(docs, embedding=embeddings, pinecone_api_key=PINECONE_API_KEY, index_name=PINECONE_INDEX_NAME)

In [15]:
# from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
embeddings = OpenAIEmbeddings()
# os.environ['PINECONE_API_KEY'] = PINECONE_INDEX_NAME
docsearch = PineconeVectorStore(index_name=PINECONE_INDEX_NAME, embedding=embeddings, pinecone_api_key=PINECONE_API_KEY)
# pc = Pinecone(api_key=PINECONE_API_KEY)
# docsearch = pc.Index(PINECONE_INDEX_NAME)

In [36]:
openai_llm = ChatOpenAI(temperature=0.4)

In [38]:
import requests
from langchain_core.tools import tool
from typing import Annotated
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

# Define Pinecone tool
@tool
def pinecone_search(query: Annotated[str, "The query to search in Pinecone."]):
    """
    Search the Pinecone vector database with the provided query.
    
    Args:
    query (str): The query to search in Pinecone.
    
    Returns:
    str: The top result from the Pinecone search.
    """
    response = docsearch.similarity_search(query)
    return response

# Define OpenWeather tool
@tool
def get_weather(city: Annotated[str, "The city to get weather information for."]):
    """
    Get the weather information for the specified city using the OpenWeather API.
    
    Args:
    city (str): The city to get weather information for.
    
    Returns:
    dict: The weather information for the specified city.
    """
    api_key = OPENWEATHER_API_KEY
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = base_url + "appid=" + api_key + "&q=" + city
    response = requests.get(complete_url)
    return response.json()

@tool
def dummy_tool(inputs):
  """
  You are given one question and you have to extract city name/name of place from it
  Don't respond anything except the city name and don't reply anything if you can't find city name and if the question is about personal information like personal business etc then response with just word business and nothing else.
  
  """
  res = openai_llm.invoke(inputs)
  return res.content



In [39]:
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
    AIMessage
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph, START
from langchain_openai import ChatOpenAI
import functools

# Helper function to create a node for a given agent
def agent_node(state, agent, name):
    result = agent.invoke(state)
    # Convert the agent output into a format that is suitable to append to the global state
    if isinstance(result, ToolMessage):
        pass
    else:
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        "sender": name,
    }

def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_tools(tools)

# Define the LLM
llm = ChatOpenAI(model="gpt-4o")

# Business agent using Pinecone
business_agent = create_agent(
    llm,
    [pinecone_search],
    system_message="You should provide business-related information."
)
business_node = functools.partial(agent_node, agent=business_agent, name="BusinessAgent")

# Weather agent using OpenWeather API
weather_agent = create_agent(
    llm,
    [get_weather],
    system_message="You should provide weather-related information."
)
weather_node = functools.partial(agent_node, agent=weather_agent, name="WeatherAgent")

# General agent using GPT-4o
general_agent = create_agent(
    llm,
    [dummy_tool],
    system_message="You should respond to general queries."
)
general_node = functools.partial(agent_node, agent=general_agent, name="GeneralAgent")



In [40]:
from typing import Annotated, Sequence, TypedDict
import operator

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

from langgraph.prebuilt import ToolNode

# Define tools
tools = [pinecone_search, get_weather, dummy_tool]
tool_node = ToolNode(tools)

# Define the router logic
from typing import Literal

def router(state) -> Literal["call_tool", "__end__", "BusinessAgent", "WeatherAgent", "GeneralAgent"]:
    messages = state["messages"]
    last_message = messages[-1]
    content = last_message.content.lower()
    if last_message.tool_calls:
        return "call_tool"
    elif "FINAL ANSWER" in last_message.content:
        return "__end__"
    elif "weather" in content:
        return "WeatherAgent"
    elif "business" in content:  # Check if the query is business-related
        return "BusinessAgent"
    else:
        return "GeneralAgent"


# Define the graph
workflow = StateGraph(AgentState)

workflow.add_node("BusinessAgent", business_node)
workflow.add_node("WeatherAgent", weather_node)
workflow.add_node("call_tool", tool_node)
workflow.add_node("GeneralAgent", general_node)

workflow.add_conditional_edges(
    "GeneralAgent",
    router,
    {"WeatherAgent": "WeatherAgent","BusinessAgent": "BusinessAgent", "call_tool": "call_tool", "__end__": END}
)

workflow.add_conditional_edges(
    "BusinessAgent",
    router,
    {"WeatherAgent": "WeatherAgent","GeneralAgent": "GeneralAgent" , "call_tool": "call_tool", "__end__": END}
)
workflow.add_conditional_edges(
    "WeatherAgent",
    router,
    {"BusinessAgent": "BusinessAgent","GeneralAgent": "GeneralAgent" , "call_tool": "call_tool", "__end__": END}
)

workflow.add_conditional_edges(
    "call_tool",
    # Each agent node updates the 'sender' field
    # the tool calling node does not, meaning
    # this edge will route back to the original agent
    # who invoked the tool
    lambda x: x["sender"],
    {
        "BusinessAgent": "BusinessAgent",
        "WeatherAgent": "WeatherAgent",
        "GeneralAgent": "GeneralAgent",
    },
)
workflow.add_edge(START, "GeneralAgent")
graph = workflow.compile()


In [20]:
# events = graph.stream(
#     {
#         "messages": [
#             HumanMessage(
#                 content="weather london"
#             )
#         ],
#     },
#     {"recursion_limit": 150},
# )
# for s in events:
#     print(s)
#     print("----")



In [42]:
while True:
    print("while 1")
    content = input("you: ")
    if content.lower() == "exit" or content.lower() == "quit":
        break
    events = graph.stream(
        {
            "messages": [
                HumanMessage(
                    content=content #"What is the weather in New York today?"
                )
            ],
        },
        {"recursion_limit": 150},
    )
    for s in events:
        print(s)
        print("----")

while 1


KeyError: 'GeneralAgent'